<a href="https://colab.research.google.com/github/LLMsLab/chat-gpt-api-lab/blob/exploration%2Fchatgpt-api-understanding/tutorial_04_chatgpt_api_with_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build your ChatGPT Clone in Python with OpenAI ChatGPT API and Gradio

In [ ]:
# Requirements
!pip install -qU python-dotenv openai gradio presidio-analyzer presidio-anonymizer
!python -m spacy download en_core_web_lg -qU

2023-05-29 01:20:31.100556: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 01:20:33.844408: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
# Importing necessary modules
import os
from dotenv import load_dotenv
import openai
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
import gradio as gr
from IPython.display import HTML, display

In [ ]:
def set_css():
    """
    Wraps the lines in the notebook's output.
    """
    display(HTML('''
    <style>
      pre {
        white-space: pre-wrap;
      }
    </style>
    '''))

get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# Mount Google Drive and ensure Google CoLab is running the correct
# version of TensorFlow
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  COLAB = True
  print("Note: using Google Colab")
  %tensorflow_version 2.x
except:
  print("Note: not using Google Colab")
  COLAB = False

Mounted at /content/drive
Note: using Google Colab
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# Load the environment variables from the .env file
load_dotenv('/content/drive/MyDrive/Projects/.env')

True

In [ ]:
# Retrieving API keys from the environment
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
def analyze_and_anonymize(text: str):
    """
    Analyze and anonymize PII data from a given text string.

    Parameters
    ----------
    text : str
        The text to be analyzed and anonymized.

    Returns
    -------
    str
        The anonymized text.

    Example
    -------
    >>> text = "Mr. Smith's phone number is 212-555-5555, his SSN is \
    >>> 432-56-5654, and his credit card number is 344078656339539"
    >>> print(analyze_and_anonymize(text))
    """
    # Set up the engine, loads the NLP module (spaCy model by default)
    # and other PII recognizers
    analyzer = AnalyzerEngine()

    # Define the entities to analyze
    entities = [
        "CREDIT_CARD",
        "CRYPTO",
        "DATE_TIME",
        "EMAIL_ADDRESS",
        "IBAN_CODE",
        "IP_ADDRESS",
        "NRP",
        "LOCATION",
        "PERSON",
        "PHONE_NUMBER",
        "MEDICAL_LICENSE",
        "URL",
        "US_BANK_NUMBER",
        "US_DRIVER_LICENSE",
        "US_ITIN",
        "US_PASSPORT",
        "US_SSN",
    ]

    # Call analyzer to get results
    results = analyzer.analyze(text=text, entities=entities, language="en")

    # Analyzer results are passed to the AnonymizerEngine for anonymization
    anonymizer = AnonymizerEngine()

    # Anonymize the text
    anonymized_text = anonymizer.anonymize(text=text, analyzer_results=results)

    return anonymized_text.text

In [ ]:
message_history = [{"role": "user", "content": f"You are an assistant."},
                   {"role": "assistant", "content": f"OK"}]

In [ ]:
def clear_history():
    global message_history
    message_history = [{"role": "user", "content": f"You are an assistant."},
                   {"role": "assistant", "content": f"OK"}]

message_history = [{"role": "user", "content": f"You are an assistant."},
                   {"role": "assistant", "content": f"OK"}]

def predict(input):
    """
    Given a user's input, this function predicts a response by sending the
    anonymized input to the OpenAI API. This prediction is then appended to the
    message history. Finally, the function returns a list of pairs of
    consecutive messages from the history, skipping the pre-prompt.

    Parameters
    ----------
    input : str
        User's original input message

    Returns
    -------
    list
        A list of tuples. Each tuple contains a pair of consecutive messages 
        from the message history.
    """
    
    # Anonymize message
    input_anonymized = analyze_and_anonymize(input)

    # Tokenize the new input sentence
    message_history.append(
        {
            "role": "user",
            "content": (
                #f"**User's original input:**\n {input}\n"
                f"**Anonymized version prepared for the " 
                f"OpenAPI request:**\n {input_anonymized}"
            ),
        }
    )

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=message_history
    )
    
    # Just the reply text
    reply_content = completion.choices[0].message.content

    message_history.append({"role": "assistant", "content": f"{reply_content}"})

    # Get pairs of msg["content"] from message history, skipping the pre-prompt:
    response = [
        (message_history[i]["content"], message_history[i + 1]["content"])
        for i in range(2, len(message_history) - 1, 2)
    ]  # Convert to tuples of list

    return response


In [ ]:
with gr.Blocks(theme="darkhuggingface") as demo:
    chatbot = gr.Chatbot()

    with gr.Row():
        txt = gr.Textbox(
            show_label=False, placeholder="Enter text and press enter"
        ).style(container=False)
    
    clear = gr.Button("New chat")
    txt.submit(predict, txt, chatbot)
    txt.submit(None, None, txt, _js="() => {''}")
    clear.click(clear_history, None, chatbot, queue=False)

#demo.launch(share=True, auth=("Stellar","AmAriLLo901"), auth_message="Check your Login details sent to your email")
demo.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:666: UserWarning: Cannot load darkhuggingface. Caught Exception: The space darkhuggingface does not exist
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6ef9981e55d482351e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
